In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
from v4vapp_backend_v2.database.db import MongoDBClient, get_mongodb_client_defaults

from v4vapp_backend_v2.config.setup import InternalConfig

target_dir = "/Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2/"
os.chdir(target_dir)
print("Current working directory:", os.getcwd())


In [ ]:
CONFIG = InternalConfig(config_filename="devhive.config.yaml").config
db_client = get_mongodb_client_defaults()

async with db_client as client:
    collection = await client.get_collection("rates")

    print("MongoDB connection string:", client.db_conn)
    print("MongoDB database name:", client.db_name)
    print("MongoDB database user:", client.db_user)
    print("MongoDB collection name:", collection.name)

    first_doc = await collection.find_one(sort=[("timestamp", 1)])
    last_doc = await collection.find_one(sort=[("timestamp", -1)])
    print("First date:", first_doc["timestamp"] if first_doc else None)
    print("Last date:", last_doc["timestamp"] if last_doc else None)
    print("Num of documents:", await collection.count_documents({}))

    # Query all btc_usd rates
    btc_usd_cursor = collection.find({"pair": "btc_usd"}, sort=[("timestamp", 1)])
    btc_usd_docs = await btc_usd_cursor.to_list(length=None)
    # Query all documents, sorted by timestamp
    cursor = collection.find({}, sort=[("timestamp", 1)])
    documents = await cursor.to_list(length=None)

    # Convert to DataFrame
    df = pd.DataFrame(documents)

    # Ensure timestamp is datetime
    if not pd.api.types.is_datetime64_any_dtype(df["timestamp"]):
        df["timestamp"] = pd.to_datetime(df["timestamp"])

    # Create figure and primary y-axis
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plot BTC/USD on left y-axis
    ax1.plot(df["timestamp"], df["btc_usd"], "b-", label="BTC/USD")
    ax1.set_xlabel("Time")
    ax1.set_ylabel("BTC/USD", color="b")
    ax1.tick_params(axis="y", labelcolor="b")

    # Create secondary y-axis
    ax2 = ax1.twinx()

    # Plot SATS/HIVE on right y-axis
    ax2.plot(df["timestamp"], df["sats_hive"], "r-", label="SATS/HIVE")
    ax2.set_ylabel("SATS/HIVE", color="r")
    ax2.tick_params(axis="y", labelcolor="r")

    # Add title and grid
    plt.title("Bitcoin Price and SATS/HIVE Over Time")
    ax1.grid(True, alpha=0.3)

    # Add legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left")

    # Layout and show plot
    fig.tight_layout()
    plt.show()

    # Close MongoDB connection
    await client.close()

In [ ]:

# Query all documents from the combined collection
collection = await client.get_collection("hive_rates")
combined_cursor = collection.find({}, sort=[("timestamp", 1)])
combined_docs = await combined_cursor.to_list(length=None)

if combined_docs:
    df = pd.DataFrame(combined_docs)
    # Ensure timestamp is datetime if not already
    if not pd.api.types.is_datetime64_any_dtype(df["timestamp"]):
        df["timestamp"] = pd.to_datetime(df["timestamp"])

    # Calculate time deltas between consecutive records
    df = df.sort_values("timestamp")
    df["delta"] = df["timestamp"].diff().dt.total_seconds().fillna(0)

    # Filter out rows where delta == 0 (ignore 0 gap records)
    df = df[df["delta"] != 0]

    print(f"Total records after filtering 0 delta: {len(df)}")

    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plot BTC/USD on left y-axis
    ax1.plot(df["timestamp"], df["btc_usd"], label="BTC/USD Rate", color="tab:blue")
    ax1.set_xlabel("Timestamp")
    ax1.set_ylabel("BTC/USD Rate", color="tab:blue")
    ax1.tick_params(axis="y", labelcolor="tab:blue")

    # Plot SATS/HIVE on right y-axis
    ax2 = ax1.twinx()
    ax2.plot(df["timestamp"], df["sats_hive"], label="SATS/HIVE Rate", color="tab:orange")
    ax2.set_ylabel("SATS/HIVE Rate", color="tab:orange")
    ax2.tick_params(axis="y", labelcolor="tab:orange")

    fig.suptitle("BTC/USD and SATS/HIVE Rates Over Time (Filtered 0 Delta)")
    fig.tight_layout()
    plt.show()
else:
    print("No data found in hive_rates_combined collection.")


In [ ]:
from datetime import datetime, timedelta, timezone

# Target datetime in UTC (assuming input is UTC)
target_dt = datetime(2025, 5, 3, 12, 45, tzinfo=timezone.utc)


# Find closest hive_usd rate
df_hive_usd = pd.DataFrame(
    [doc for doc in btc_usd_docs + sats_hive_docs if doc["pair"] == "hive_usd"]
)
if not df_hive_usd.empty:
    df_hive_usd["timedelta"] = (df_hive_usd["timestamp"] - target_dt).abs()
    closest_hive_usd_row = df_hive_usd.loc[df_hive_usd["timedelta"].idxmin()]
    print("Closest HIVE/USD rate:")
    print(closest_hive_usd_row[["timestamp", "value", "timedelta"]])
else:
    print("No data found for hive_usd.")

# Find closest hive_hbd rate
df_hive_hbd = pd.DataFrame(
    [doc for doc in btc_usd_docs + sats_hive_docs if doc["pair"] == "hive_hbd"]
)
if not df_hive_hbd.empty:
    df_hive_hbd["timedelta"] = (df_hive_hbd["timestamp"] - target_dt).abs()
    closest_hive_hbd_row = df_hive_hbd.loc[df_hive_hbd["timedelta"].idxmin()]
    print("Closest HIVE/HBD rate:")
    print(closest_hive_hbd_row[["timestamp", "value", "timedelta"]])
else:
    print("No data found for hive_hbd.")


# Find closest BTC/USD rate
df_btc_usd["timedelta"] = (df_btc_usd["timestamp"] - target_dt).abs()
closest_btc_usd_row = df_btc_usd.loc[df_btc_usd["timedelta"].idxmin()]
print("Closest BTC/USD rate:")
print(closest_btc_usd_row[["timestamp", "value"]])

# Find closest SATS/HIVE rate
df_sats_hive["timedelta"] = (df_sats_hive["timestamp"] - target_dt).abs()
closest_sats_hive_row = df_sats_hive.loc[df_sats_hive["timedelta"].idxmin()]
print("Closest SATS/HIVE rate:")
print(closest_sats_hive_row[["timestamp", "value"]])

# Optionally, drop the helper column if you don't need it
df_btc_usd.drop(columns=["timedelta"], inplace=True)
df_sats_hive.drop(columns=["timedelta"], inplace=True)

In [ ]:
import asyncio

from pymongo import AsyncMongoClient

old_client = MongoDBClient(
    db_conn=CONFIG.dbs_config.default_connection,
    db_name=CONFIG.dbs_config.default_name,
    db_user=CONFIG.dbs_config.default_user,
)

new_client = AsyncMongoClient(old_client.uri, tz_aware=True)
db = new_client[CONFIG.dbs_config.default_name]
collection = db["hive_rates"]
doc = await collection.find_one({"pair": "btc_usd"})


async def find_nearest_pair_rate(collection, pair, target_dt):
    """
    Find the document for the given pair with the timestamp nearest to target_dt.

    Args:
        collection: AsyncCollection instance.
        pair (str): The pair to search for (e.g., 'btc_usd').
        target_dt (datetime): The target datetime (must be timezone-aware).

    Returns:
        dict or None: The nearest document, or None if not found.
    """
    # Find the document just before or at target_dt
    pairs = await collection.distinct("pair")
    async with asyncio.TaskGroup() as tg:
        pair_tasks = {
            p: tg.create_task(
                asyncio.gather(
                    collection.find_one(
                        {"pair": p, "timestamp": {"$lte": target_dt}}, sort=[("timestamp", -1)]
                    ),
                    collection.find_one(
                        {"pair": p, "timestamp": {"$gt": target_dt}}, sort=[("timestamp", 1)]
                    ),
                )
            )
            for p in pairs
        }
        # Remove the parallel search for all pairs, only keep the logic for the requested pair
        before_task = tg.create_task(
            collection.find_one(
                {"pair": pair, "timestamp": {"$lte": target_dt}}, sort=[("timestamp", -1)]
            )
        )
        after_task = tg.create_task(
            collection.find_one(
                {"pair": pair, "timestamp": {"$gt": target_dt}}, sort=[("timestamp", 1)]
            )
        )
    before = before_task.result()
    after = after_task.result()

    # Choose the closest one
    if before and after:
        delta_before = abs(before["timestamp"] - target_dt)
        delta_after = abs(after["timestamp"] - target_dt)
        return before if delta_before <= delta_after else after
    return before or after


target_dt = datetime(2025, 5, 21, 6, 4, tzinfo=timezone.utc)

# Example usage:
nearest_btc_usd = await find_nearest_pair_rate(collection, "btc_usd", target_dt)
print(
    "Nearest BTC/USD document:",
    nearest_btc_usd,
    "\n\nTimestamp:",
    nearest_btc_usd["timestamp"] if nearest_btc_usd else None,
    "\n\nTimedelta:",
    abs(nearest_btc_usd["timestamp"] - target_dt) if nearest_btc_usd else None,
)

In [ ]:
import asyncio
import random
from timeit import default_timer as timer

# Generate 10,000 random datetimes within the last 3 months
now = datetime.now(timezone.utc)
three_months_ago = now - timedelta(days=90)
random_dates = [
    three_months_ago
    + timedelta(seconds=random.randint(0, int((now - three_months_ago).total_seconds())))
    for _ in range(10000)
]

start_time = timer()
tasks = [find_nearest_pair_rate(collection, "btc_usd", dt) for dt in random_dates]
results = await asyncio.gather(*tasks)
elapsed = timer() - start_time

print(f"Elapsed time for 10,000 calls with random dates: {elapsed:.2f} seconds")

In [ ]:
# Get all unique 'pair' values in the collection
available_pairs = await collection.distinct("pair")
print("Available pairs:", available_pairs)


In [ ]:
# Combine all four quotes (btc_usd, sats_hive, hive_usd, hive_hbd) into a single record per timestamp
# Only btc_usd and sats_hive are available as DataFrames, hive_usd and hive_hbd are empty
# We'll join on timestamp and output a new list of dicts

# First, merge btc_usd and sats_hive on timestamp
combined_df = pd.merge(
    df_btc_usd[["timestamp", "value"]].rename(columns={"value": "btc_usd"}),
    df_sats_hive[["timestamp", "value"]].rename(columns={"value": "sats_hive"}),
    on="timestamp",
    how="outer",
)

# If you have hive_usd and hive_hbd data, merge them as well (currently empty)
# Example:
# combined_df = combined_df.merge(
#     df_hive_usd[["timestamp", "value"]].rename(columns={"value": "hive_usd"}),
#     on="timestamp", how="outer"
# ).merge(
#     df_hive_hbd[["timestamp", "value"]].rename(columns={"value": "hive_hbd"}),
#     on="timestamp", how="outer"
# )

# For now, add empty columns for hive_usd and hive_hbd
combined_df["hive_usd"] = None
combined_df["hive_hbd"] = None

# Sort by timestamp
combined_df.sort_values("timestamp", inplace=True)

# Write back as new records (one per timestamp, with all four quotes)
new_records = []
for _, row in combined_df.iterrows():
    record = {
        "timestamp": row["timestamp"],
        "btc_usd": row["btc_usd"],
        "sats_hive": row["sats_hive"],
        "hive_usd": row["hive_usd"],
        "hive_hbd": row["hive_hbd"],
    }
    new_records.append(record)

# Example: insert into a new collection
# await db["hive_rates_combined"].insert_many(new_records)

print("Sample combined record:", new_records[0])
print(f"Total combined records: {len(new_records)}")

In [ ]:
# Bulk insert new_records into the "hive_rates" timeseries collection
# Assumes new_records is a list of dicts as constructed previously

result = await db["hive_rates"].insert_many(new_records)
print(f"Inserted {len(result.inserted_ids)} records into hive_rates timeseries collection.")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

collection = db["hive_rates_combined"]
# Query all documents from the combined collection
combined_cursor = collection.find({}, sort=[("timestamp", 1)])
combined_docs = await combined_cursor.to_list(length=None)

if combined_docs:
    df = pd.DataFrame(combined_docs)
    # Ensure timestamp is datetime if not already
    if not pd.api.types.is_datetime64_any_dtype(df["timestamp"]):
        df["timestamp"] = pd.to_datetime(df["timestamp"])

    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Plot BTC/USD on left y-axis
    ax1.plot(df["timestamp"], df["btc_usd"], label="BTC/USD Rate", color="tab:blue")
    ax1.set_xlabel("Timestamp")
    ax1.set_ylabel("BTC/USD Rate", color="tab:blue")
    ax1.tick_params(axis="y", labelcolor="tab:blue")

    # Plot SATS/HIVE on right y-axis
    ax2 = ax1.twinx()
    ax2.plot(df["timestamp"], df["sats_hive"], label="SATS/HIVE Rate", color="tab:orange")
    ax2.set_ylabel("SATS/HIVE Rate", color="tab:orange")
    ax2.tick_params(axis="y", labelcolor="tab:orange")

    fig.suptitle("BTC/USD and SATS/HIVE Rates Over Time (Combined Collection)")
    fig.tight_layout()
    plt.show()
else:
    print("No data found in hive_rates_combined collection.")
